##### <font color='darkblue'>Week 6 - Lab Notebook - Working with Biological Databases</font>

- October 2023
- https://https://github.com/tisimpson/bioinformatics1
- ian.simpson@ed.ac.uk

##### <font color='darkblue'>Introduction</font>
In this computing lab we are going to be making some plots and tables. We've included code to make simple plots using matplotlib and nicer looking tables using PrettyTable in the various notebooks in the previous weeks so you can borrow code from there and modify it as needed.

In this computing lab you are going to try to find answers to biological questions using data that you query/download/analyse from some of the websites we introduced last week. You can follow the code in the week5&6 notebooks and make changes to them and use some time to tackle some of the challenges/problems at the end of the notebooks. We've summarised these below.

##### <font color='darkblue'>Learning Outcomes</font>
After this tutorial you should be comfortable with:
• Identifying a selection of databases likely to contain data you need in a research project
• Navigating and creating custom queries to extract and download data from these databases
• Processing & Filtering data to use in summary analysis and visualisation

##### <font color='darkblue'>Activity 1 – Genomic Data</font>

Refer to the [bio1_week5&6_part1.ipynb](https://github.com/tisimpson/bioinformatics1/blob/main/labs/notebooks/bio1_week5%266_part1.ipynb) notebook for this activity. If you have already worked through this notebook skip to number 4.

Go to the [NCBI download page](https://www.ncbi.nlm.nih.gov/home/download/) and click "download by FTP". This will lead to a directory tree. Click "gene", then "data" and either fetch the complete "gene_info.gz" file or better navigate further into the "GENE_INFO" directory and from the directories inside there find the gene_info file for the individual species (a much smaller file). For example the human file is found [here](https://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz). Have a look at the README file in this directory for detailed information about the content and structure of the files.

- Human (Homo sapiens)
- Mouse (Mus musculus)
- Rat (Rattus Norvegicus)
- Fruitfly (Drosophila melanogaster)
- Yeast (Saccharomyces cerevisiae)
  
1. Using these data files create plots for each species to show:
   - Total Number of Genes by Species (in one plot)
   - Number of Genes by gene_type (you can try individual plots and stacked histograms)
2.  For the same species find the total size of their genomes (in nucleotides) (you can find this by finding the genomes in the genome browsing table at [NCBI Genomes](https://www.ncbi.nlm.nih.gov/datasets/genomes/?taxon=overview)).
3. Now create a plot where you normalise the number of genes by genome size and plot this for all the species on one plot, what do you think of the outcome?
4. (optional) Can you perform the same analysis as 1-3 above, but for the number of unique transcripts in each genome? You are going to want to work with the feature table file which summarises the information for genomic features.

In [ ]:
# Activity 1.1

import pandas as pd

# load the 5 genome files from .../data/genomes
# and store them in a list called genomes

#read the gene_info files into Pandas data frames
human_df = pd.read_csv('../data/genomes/Homo_sapiens.gene_info.gz', compression='gzip', header=0, sep='\t')
mouse_df = pd.read_csv('../data/genomes/Mus_musculus.gene_info.gz', compression='gzip', header=0, sep='\t')
rat_df = pd.read_csv('../data/genomes/Rattus_norvegicus.gene_info.gz', compression='gzip', header=0, sep='\t')
fruitfly_df = pd.read_csv('../data/genomes/Drosophila_melanogaster.gene_info.gz', compression='gzip', header=0, sep='\t')
yeast_df = pd.read_csv('../data/genomes/Saccharomyces_cerevisiae.gene_info.gz', compression='gzip', header=0, sep='\t')

In [ ]:
# Activity 1.1

# use seaborn to plot gene count by species
import seaborn as sns
genomes = {'Homo sapiens':human_df,'Mus musculus':mouse_df,'Rattus norvegicus':rat_df,'Drosophila melanogaster':fruitfly_df,'Saccharomyces cerevisiae':yeast_df}

# create a list of tuples containing species name and gene count
gene_counts = [(species, len(genomes[species].value_counts('Symbol'))) for species in genomes]

# create a dataframe from the list of tuples
df_gene_counts = pd.DataFrame(gene_counts, columns=['Species', 'Gene Count'])

# plot the gene count by species using a barplot
ax = sns.barplot(x='Species', y='Gene Count', data=df_gene_counts)
ax.set_xticks(ax.get_xticks(), labels=df_gene_counts['Species'],rotation=30,ha='right');

In [ ]:
# now create a stack barplot of the gene count by gene type_of_gene
# first create a list of tuples containing species name, gene type, and gene count
# exclude any entries for which gene_type is either unknown, other, or biological-region
gene_counts = [(species, gene_type, len(genomes[species][genomes[species]['type_of_gene'] == gene_type])) for species in genomes for gene_type in genomes[species]['type_of_gene'].unique() if gene_type not in ['unknown', 'other', 'biological-region']]

# create a dataframe from the list of tuples
df_gene_counts = pd.DataFrame(gene_counts, columns=['Species', 'Gene Type', 'Gene Count'])

# plot this using a stacked barplot with species on the x-axis and gene count on the y-axis rotated 30 degrees
ax = sns.barplot(x='Species', y='Gene Count', hue='Gene Type', data=df_gene_counts)
ax.tick_params(axis='x', labelrotation = 40)


In [ ]:
# Actitivy 1.2

# ul.request.urlretrieve('https://ftp.ncbi.nlm.nih.gov/genomes/GENOME_REPORTS/overview.txt','../data/genomes/genomes.txt')

#build the genomes report dataframe
genomes_df = pd.read_csv('../data/genomes/genomes.txt',header=0,sep='\t',low_memory=False)

# find the rows matching our species of interest
species = ['Homo sapiens','Mus musculus','Rattus norvegicus','Drosophila melanogaster','Saccharomyces cerevisiae']

# find the rows in genomes_df matching our species
species_rows = genomes_df[genomes_df['#Organism/Name'].isin(species)]
# print the rows
species_rows.head()

In [ ]:
# Activity 1.3

gene_merge = pd.merge(df_gene_counts,species_rows,left_on='Species',right_on='#Organism/Name')

# divide 'Gene Count' column by 'Size (Mb)' column and add to dataframe
gene_merge['Gene Density'] = gene_merge['Gene Count'] / pd.to_numeric(gene_merge['Size (Mb)'])

# plot this using a stacked barplot with species on the x-axis and gene count on the y-axis rotated 30 degrees
ax = sns.barplot(x='Species', y='Gene Density', hue='Gene Type', data=gene_merge)
ax.tick_params(axis='x', labelrotation = 40)



In [ ]:
# Activity 1.4

import urllib as ul

# number of unique transcripts per species
# normalise by genome size

ref_locations = ul.request.urlretrieve('https://ftp.ncbi.nlm.nih.gov/genomes/refseq/assembly_summary_refseq.txt','../data/genomes/refseq_locs.txt');

# compress the file with gzip compression
import gzip
with open('../data/genomes/refseq_locs.txt', 'rb') as f_in:
    with gzip.open('../data/genomes/refseq_locs.txt.gz', 'wb') as f_out:
        f_out.writelines(f_in)
        
# remove the uncompressed file
import os
os.remove('../data/genomes/refseq_locs.txt')


In [ ]:
df_ref_locations = pd.read_csv('../data/genomes/refseq_locs.txt.gz',compression='gzip',sep='\t',header=1,low_memory=False)

df_ref_locations.head()

In [ ]:
# find the rows matching our species of interest

# best searched by taxonmic ids

tax_ids = [9606,10090,10116,7227,4932]

# find the rows in genomes_df matching our species and return the organism_name and taxid columns
# this is a bit fiddly

species_rows = df_ref_locations[
    df_ref_locations['species_taxid'].isin(tax_ids) &
    ((df_ref_locations['refseq_category'] == 'reference genome')
    | (df_ref_locations['refseq_category'] == 'representative genome'))
][['organism_name','taxid','ftp_path']]

# print the rows
species_rows.head()

In [ ]:
# get the assembly report files for each species and name the files after the species
for index, row in species_rows.iterrows():
    ul.request.urlretrieve(row['ftp_path'] + '/' + row['ftp_path'].split('/')[-1] + '_feature_table.txt.gz','../data/genomes/' + row['organism_name'].replace(' ','_') + '_feature_table.txt.gz')

In [ ]:
# load the feature tables into dataframes

def feature_types(df_features):
    featureCounts = df_features['# feature'].value_counts().to_frame(name='counts')
    featureCounts = featureCounts.rename_axis('feature_type').reset_index()
    #rename the columns
    featureCounts.columns = ['feature_type','counts']
    return featureCounts


#create a dataframe containing the feature counts for each feature type
feature_counts = {}
for index, row in species_rows.iterrows():
    feature_counts[row['organism_name']] = feature_types(pd.read_csv('../data/genomes/' + row['organism_name'].replace(' ','_') + '_feature_table.txt.gz',sep='\t',header=0,compression='gzip',low_memory=False))

In [ ]:
# add the species to each row
for species in feature_counts:
    feature_counts[species] = feature_counts[species].assign(Species=species)
    
# concatenate the dataframes
df_feature_counts = pd.concat(feature_counts)

# plot the feature counts by species using a stacked barplot
ax = sns.barplot(x='Species', y='counts', hue='feature_type', data=df_feature_counts)
ax.set_ylabel('Feature Count')
ax.tick_params(axis='x', labelrotation = 40)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

In [ ]:
# now correct for genome size using the gene_merge dataframe
# first merge the feature_counts dataframe with the gene_merge dataframe

df_feature_counts = pd.merge(df_feature_counts,gene_merge,left_on='Species',right_on='Species')

# divide the feature counts by the genome size
df_feature_counts['counts'] = df_feature_counts['counts'] / pd.to_numeric(df_feature_counts['Size (Mb)'])

# plot the feature counts by species using a stacked barplot
ax = sns.barplot(x='Species', y='counts', hue='feature_type', data=df_feature_counts)
ax.set_ylabel('Feature Density')
ax.tick_params(axis='x', labelrotation = 40)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

##### <font color='darkblue'>Activity 2 – Pathways and Protein-Protein Interaction Networks</font>

Refer to the [bio1_week5&6_part2.ipynb](https://github.com/tisimpson/bioinformatics1/blob/main/labs/notebooks/bio1_week5%266_part2.ipynb) notebook for this activity.

1. Go to the [KEGG pathway database](https://www.genome.jp/kegg/pathway.html) and download a list of human genes that are part of the “Cell adhesion molecules". In the search make sure you restrict it to humans only. This is actually pretty tricky so if you get stuck the list can be found [here](https://www.genome.jp/entry/pathway+hsa04514) and [here](https://www.genome.jp/dbget-bin/get_linkdb?-t+genes+path:hsa04514).

We're going to look for evidence of interactions between these proteins using the StringDB database which we will look at in more detail in a couple of weeks when we learn about networks and their analysis. In the week 8 lab we're going to learn how to do the whole process computationally.

2. Parse that list (the numeric part of the hsa : <NUMBER> identifier in the list is in fact the NCBI Entrez GeneID accession) and then go to [StringDB](https://string-db.org) click "Search" and then on the left-hand side "Multiple Proteins" and paste in the search list being sure to select human. Use the tabs to find the information to answer the following questions.
3. How many interactions are there? How many orphans (proteins with no interactions) are there?
4. What is the average number of interactions that any protein in the list has with other members (this is called the mean degree)?
5. Now repeat 2-4 above several times, each time applying these different restrictions using the "Settings" tab with the following active interaction data sources:
   - Experimental, Co-expression, and Co-occurence
   - Experimental only
What difference do these changes make to the number of interactions and the mean degree? Why do you think this is?

**NB** The 3 activities below are much easier to gather via Python coding. This week (look also at the notebooks we will study on Friday) you will see how programmatic access greatly facilitates many of the tasks we would like to perform.

In [ ]:
# Activity 2.1

# Fetching KEGG pathway data

# Now we will use this file which contains the full pathway details including the gene names.

# open the file
cam_file = open('../data/pathways/cams.txt','r')

# create an empty dataframe with two columns
cam_df = pd.DataFrame(columns=['gene_id','gene_symbol','description'])

# set a flag for our parser
flag=0

# work through the text file one line at a time
for line in cam_file:
    # find the start of the gene entries
    if 'GENE' in line:
        # add the first gene tp the dataframe
        gene_id,remain = line.strip('GENE').strip().split('  ')
        gene_symbol,description = remain.split(';')
        # add a new row to the dataframe containing the gene_id and description
        cam_df = pd.concat([cam_df,pd.DataFrame([[gene_id,gene_symbol,description]],columns=['gene_id','gene_symbol','description'])],ignore_index=True)
        # set the flag to 1, we are in the gene section of the file
        flag = 1
    # stop when we reach the end of the section and escape the file
    elif 'REFERENCE' in line:
        break
    # continue adding the genes to the dataframe
    elif flag == 1:
        try:
            gene_id,remain = line.strip('GENE').strip().split('  ')
            gene_symbol,description = remain.split(';')
            # add the gene to the dataframe
            cam_df = pd.concat([cam_df,pd.DataFrame([[gene_id,gene_symbol,description]],columns=['gene_id','gene_symbol','description'])],ignore_index=True)
        except:
            pass
# close the file
cam_file.close()

# view the file
cam_df.head()

# you now have the gene_ids (NCBI EntrezIDs for the genes in the pathway)
print('The Cell Adhesion Molecules pathway has '+str(cam_df.shape[0])+' genes in it.\n')

gene_symbols = cam_df['gene_symbol'].to_numpy()

# show the gene_symbols
print(gene_symbols)

In [ ]:
# Activity 2.2 and 2.3

# create a concatenated list of entrezIDs as strings
# note we are taking integer gene_ids from the 'gene_id' column of the dataframe we generated above then using
# the map function to convert each one into a string. The join function then concatenates them using the '%0D' string
# to stitch them all together. This string will be used to help us build the API query URL.
entrezIDs = '%0D'.join(map(str,cam_df['gene_id']))

# pass the list of EntrezIDs to the String-DB API return the String-IDs
# we first form the query url using the 'get_string_ids' API function which takes a list of identifiers and
# converts them into the internal String-DB accession IDs. This massively speeds up the search and allows us to
# search for more than 10 at once which is an API restriction for other API functions if String-DB internal accessions 
# aren't used.
query_url = 'https://string-db.org/api/tsv-no-header/get_string_ids?identifiers='+entrezIDs+'&species=9606&format=only-ids'

# use the urllib library to retrieve the String-DB internal IDs
result = ul.request.urlopen(query_url).read().decode('utf-8')

# now we want to query String-DB to retrieve interactions from this list of String-DB IDs
# we create a concatenated list of stringdbIDs in much the same way as above for the Entrez Gene IDs
stringdbIDs = '%0D'.join(result.splitlines())

# again we build the query for interactions using the String-DB IDs
query_url = 'https://string-db.org/api/tsv/network?identifiers='+stringdbIDs+'&species=9606'

# again using urllib to retrieve the interactions these are returned in a standard tab delimied text format
interactions = ul.request.urlopen(query_url).read().decode('utf-8').splitlines()

# we need to split the result by these 'tabs' (\t - is used to identfy them)
int_test = [interaction.split('\t') for interaction in interactions]

# we extract the field names from the first row
column_names = int_test[:1][0]

# create a Pandas dataframe of the interaction data we have just retrieved from String-DB
interactions_df = pd.DataFrame(int_test,columns=column_names)

# delete the first row that held the fieldnames but we no longer need
interactions_df = interactions_df.drop(labels=0,axis=0)

# remove any duplicate rows
final_interactions = interactions_df.drop_duplicates()

# show the top of the protein-protein interaction table
final_interactions.head()

In [ ]:
# Activity 2.3
# check which genes are missing from the interaction table

# find the unique gene symbols in the interaction table
unique_genes = set(final_interactions['preferredName_A']) | set(final_interactions['preferredName_B'])

# compare to the gene symbols in the pathway
missing_genes = set(gene_symbols) - set(unique_genes)

# print the missing genes
print('The following genes are missing from the interaction table and are therefore orphans:')
print(missing_genes)

In [ ]:
# Activity 2.4
# create a network from the interaction table

import networkx as nx

# create an empty graph
G = nx.Graph()

# add the nodes to the graph
G.add_nodes_from(unique_genes)

# add the edges to the graph
G.add_edges_from(final_interactions[['preferredName_A','preferredName_B']].to_numpy())

# how many edges and nodes in the graph?
print('The graph has '+str(G.number_of_edges())+' edges and '+str(G.number_of_nodes())+' nodes.')

# divide the number of edges by the number of nodes to get the average degree
print('The average degree of the graph is '+str(G.number_of_edges()/G.number_of_nodes())+'.\n')

In [ ]:
# Activity 2.5

# query string again but restrict to only experimentally determined interactions
query_url = 'https://string-db.org/api/tsv/network?identifiers='+stringdbIDs+'&species=9606'

# again using urllib to retrieve the interactions these are returned in a standard tab delimied text format
interactions = ul.request.urlopen(query_url).read().decode('utf-8').splitlines()

# we need to split the result by these 'tabs' (\t - is used to identfy them)
int_test = [interaction.split('\t') for interaction in interactions]

# we extract the field names from the first row
column_names = int_test[:1][0]

# create a Pandas dataframe of the interaction data we have just retrieved from String-DB
interactions_df = pd.DataFrame(int_test,columns=column_names)

# delete the first row that held the fieldnames but we no longer need
interactions_df = interactions_df.drop(labels=0,axis=0)

# remove any duplicate rows
final_interactions = interactions_df.drop_duplicates()

final_interactions.head()


In [ ]:

# restrict to experimentally determined interactions by filtering on the 'escore' column > 0
final_interactions['escore'] = pd.to_numeric(final_interactions['escore'])

# filter the dataframe to only include interactions with an escore > 0
final_interactions_experimental = final_interactions[final_interactions['escore'] > 0]

# check which genes are missing from the interaction table

# find the unique gene symbols in the interaction table
unique_genes = set(final_interactions_experimental['preferredName_A']) | set(final_interactions_experimental['preferredName_B'])

# compare to the gene symbols in the pathway
missing_genes = set(gene_symbols) - set(unique_genes)

# print the missing genes
print('There are '+str(len(missing_genes))+' missing genes')
print('The following genes are missing from the interaction table and are therefore orphans:')
print(missing_genes)

# create an empty graph
G = nx.Graph()

# add the nodes to the graph
G.add_nodes_from(unique_genes)

# add the edges to the graph
G.add_edges_from(final_interactions_experimental[['preferredName_A','preferredName_B']].to_numpy())

# how many edges and nodes in the graph?
print('The graph has '+str(G.number_of_edges())+' edges and '+str(G.number_of_nodes())+' nodes.')

# divide the number of edges by the number of nodes to get the average degree
print('The average degree of the graph is '+str(G.number_of_edges()/G.number_of_nodes())+'.\n')


##### <font color='darkblue'>Activity 3 - Gene Ontologies</font>

Refer to the [bio1_week5&6_part3.ipynb](https://github.com/tisimpson/bioinformatics1/blob/main/labs/notebooks/bio1_week5%266_part3.ipynb) notebook for this activity.

1. Using the same list of genes as used in the Protein-Protein interaction example above first download the [gene2GO.gz](https://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz) file that contains mappings between the genes (NB that in effect we are treating genes and proteins as the same here, but remember that technically genes can code for more than one protein isoform and the different isoforms could well have slightly different functions).

We will be learning a lot more about ontologies next week, but you can find out more about the Gene Ontology [here](http://geneontology.org).

2. This file contains ALL gene->gene-ontology-term mappings for all species so you need to restrict this to mappings for human genes only.
3. Map the genes from the protein-protein interaction list to the identifier in the gene2go file to identify all of the GO annotations for each gene. Note that you will need to create a unique list of proteins in the protein-protein interaction list so:
    - A-B, A-C, A-D (interactions) becomes A, B, C, D to look for mappings.
4. What is the most common annotation for genes in the list?
5. How many times is that most frequent annotated term found amongst all genes in the human genome?
6. Is that term found more times than you would expect in the set of genes from the protein-protein interaction data? Why am I asking this!?

In [ ]:
# Activity 3.1-3.6

# The solutions are in the notebbok `bio1_week5&6_part3.ipynb`

##### <font color='darkblue'>Activity 4 – Searching PubMed</font>

Refer to the [bio1_week5&6_part4.ipynb](https://github.com/tisimpson/bioinformatics1/blob/main/labs/notebooks/bio1_week5%266_part4.ipynb) notebook for this activity.

Practice using [search field tags](https://pubmed.ncbi.nlm.nih.gov/help/#using-search-field-tags) including MeSH limits to explore some of these questions at [NCBI PubMed](https://pubmed.ncbi.nlm.nih.gov)
1. Plot a graph of how many papers were in PubMed in each year for the last 10 years
2. How many papers are there relating to Cadherin-7 and human disease? what diseases are mentioned?
    - Try three different search strategies, which one do you think is best and why?
3. Can you use PubMed to work out how the emerging popularity of single-cell RNA sequencing to measure gene expression over the last 10 years?

In [ ]:
# Activity 4.1-4.3

# The solutions are in the notebook `bio1_week5&6_part4.ipynb`